In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
from flask import Markup

In [2]:
#Scarping News
url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
response=requests.get(url)
soup=BeautifulSoup(response.text,"html.parser")
n_title=soup.find("div", class_="content_title").a.text.strip('\n')
n_pgraph=soup.find("div", class_="rollover_description_inner").text.strip('\n')


In [3]:
#Adding variables to a Dictionary
mars={
    "Title":n_title,
    "Description":n_pgraph
}

In [19]:
#Scraping Images
executable_path={"executable_path":ChromeDriverManager().install()}
browser=Browser("chrome", **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\jv91250\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [20]:
#My computer has a internet service controlled by my company, I can´t use the "Robot" tool for scrappe, but here would be the code
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")
html=browser.html
soup=BeautifulSoup(html,"html.parser")

In [33]:
#Using regular scrapping with Get method
url_i="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
response_i=requests.get(url_i)
soup=BeautifulSoup(response_i.text,"html.parser")
image_url = soup.find("div", class_="carousel_items").a["data-fancybox-href"]

/spaceimages/images/mediumsize/PIA23354_ip.jpg


In [34]:
featured_image_url=f"https://www.jpl.nasa.gov{image_url}"
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA23354_ip.jpg'

In [35]:
mars["imageURL"]=featured_image_url

In [36]:
#Scrapping Facts
facts_url="https://space-facts.com/mars/"
facts=pd.read_html(facts_url)
facts_table=facts[0].copy()
facts_table=facts_table.rename(columns={0:"Title",1:"Value"})
facts_table_html=Markup(facts_table.to_html())

In [37]:
mars["facts_table"]=facts_table_html

In [60]:
#Scrapping Hemispheres
hemispheres_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
response=requests.get(hemispheres_url)
soup=BeautifulSoup(response.text,"html.parser")
hemispheres=soup.find_all("div",class_="item")

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/><div class="description"><h3>Cerberus Hemisphere Enhanced</h3></div></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/><div class="description"><h3>Schiaparelli Hemisphere Enhanced</h3></div></a><spa

In [66]:
mars_hemispheres=[]
for h in hemispheres:
    img_url_aux=h.a.img["src"]
    h_description=h.div.h3.text.rstrip(" Enhanced")
    mars_hemispheres.append(
    {
        "Hemisphere":h_description,
        "URL":f"https://astrogeology.usgs.gov/{img_url_aux}"
    }
    )

[{'Hemisphere': 'Cerberus Hemispher', 'URL': 'https://astrogeology.usgs.gov//cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'}, {'Hemisphere': 'Schiaparelli Hemispher', 'URL': 'https://astrogeology.usgs.gov//cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png'}, {'Hemisphere': 'Syrtis Major Hemispher', 'URL': 'https://astrogeology.usgs.gov//cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png'}, {'Hemisphere': 'Valles Marineris Hemispher', 'URL': 'https://astrogeology.usgs.gov//cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png'}]


In [67]:
mars["hemispheres"]=mars_hemispheres